In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
%pip install torchbnn

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

data = pd.read_csv('CombinedData.csv')
stop_words = set(stopwords.words('english'))




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
glove_path = '/content/drive/MyDrive/glove.6B.300d.txt'


Mounted at /content/drive


In [ ]:
from textblob import TextBlob
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
nltk.download('punkt')
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
import string
exclude = string.punctuation
nltk.download('punkt_tab')
stop_words = set(stopwords.words('english'))
def clean_text(text):
    # Lowering letters
    text = text.lower()
    # Removing html tags
    text = re.sub('<[^>]*>', '', text)
    # Removing emails
    text = re.sub('\S*@\S*\s?', '', text)
    # Removing urls
    text = re.sub('https?://[A-Za-z0-9]','',text)
    # Removing numbers
    text = re.sub('[^a-zA-Z]',' ',text)
    word_tokens = word_tokenize(text)
    filtered_sentence = []
    for word_token in word_tokens:
        if word_token not in stop_words:
            filtered_sentence.append(word_token)

    # Joining words
    text = (' '.join(filtered_sentence))
    #print(len(text))
    return text


data_cleaned = data[['statement', 'status']].dropna()

data_cleaned['statement'] = data_cleaned['statement'].apply(clean_text)
data_cleaned = data_cleaned[['statement', 'status']].dropna(subset=['statement'])



#data_cleaned['statement'] = data_cleaned['statement'].apply(lambda x: remove_stopwords(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from nltk.tokenize import word_tokenize
data = data_cleaned[['statement', 'status']]
# Load GloVe embeddings from file
def load_glove_embeddings(glove_file_path, embedding_dim=300):
    embeddings = {}
    with open(glove_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = torch.tensor([float(val) for val in values[1:]], dtype=torch.float)
            embeddings[word] = vector
    return embeddings

# Replace with the path to your GloVe file

glove_embeddings = load_glove_embeddings(glove_path)

# Tokenizer function
def tokenize(text):
    return text.lower().split()

# Encode each sentence with GloVe embeddings
def encode_sentence(sentence):
    tokens = tokenize(sentence)
    embeddings = [glove_embeddings[token] for token in tokens if token in glove_embeddings]
    return torch.stack(embeddings) if embeddings else torch.zeros((1, 300))

data['embeddings'] = data['statement'].apply(encode_sentence)
# Preprocess the data

label_encoder = LabelEncoder()
data['status'] = label_encoder.fit_transform(data['status'])


# Split into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

# Custom Dataset
class TextDataset(Dataset):
    def __init__(self, data):
        self.embeddings = data['embeddings']
        self.labels = data['status']

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

# Collate function to pad sequences
def collate_fn(batch):
    texts, labels = zip(*batch)
    texts_padded = pad_sequence(texts, batch_first=True)
    labels = torch.tensor(labels, dtype=torch.long)
    return texts_padded, labels

train_dataset = TextDataset(train_data)
test_dataset = TextDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)
device = torch.device("cpu")





In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# Define LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim, n_layers=1, bidirectional=True, dropout=0.5):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        if self.lstm.bidirectional:
            hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)
        else:
            hidden = hidden[-1]
        hidden = self.dropout(hidden)
        output = self.fc(hidden)
        return output

# Model parameters
embedding_dim = 300
hidden_dim = 128
output_dim = len(label_encoder.classes_)
model = LSTMClassifier(embedding_dim, hidden_dim, output_dim).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Function to train the model
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()

    total_loss = 0
    for texts, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        texts, labels = texts.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    return total_loss / len(train_loader)

# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for texts, labels in test_loader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            predictions = torch.argmax(outputs, dim=1)
            total_correct += (predictions == labels).sum().item()
            total_samples += labels.size(0)
    accuracy = total_correct / total_samples
    return accuracy

# Function to test the model on a sample input
def test_sample(model, text):
    model.eval()
    with torch.no_grad():
        embedding = encode_sentence(text).unsqueeze(0).to(device)
        output = model(embedding)
        prediction = torch.argmax(output, dim=1).item()
    return label_encoder.inverse_transform([prediction])[0]



num_epochs = 30
for epoch in range(num_epochs):
    train_loss = train_model(model, train_loader, criterion, optimizer,num_epochs)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}")

    # Evaluate on test data
    test_accuracy = evaluate_model(model, test_loader)
    print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Training Epoch 1: 100%|██████████| 1317/1317 [00:19<00:00, 68.13it/s]


Epoch [1/30], Loss: 0.9431
Test Accuracy: 72.10%


Training Epoch 2: 100%|██████████| 1317/1317 [00:19<00:00, 68.35it/s]


Epoch [2/30], Loss: 0.7196
Test Accuracy: 74.93%


Training Epoch 3: 100%|██████████| 1317/1317 [00:19<00:00, 68.16it/s]


...

Epoch [28/30], Loss: 0.0996
Test Accuracy: 74.65%


Training Epoch 29: 100%|██████████| 1317/1317 [00:19<00:00, 66.81it/s]


Epoch [29/30], Loss: 0.0951
Test Accuracy: 74.95%


Training Epoch 30: 100%|██████████| 1317/1317 [00:19<00:00, 67.99it/s]


Epoch [30/30], Loss: 0.0936
Test Accuracy: 74.88%


In [ ]:
import torch

# Assuming 'model' is your trained LSTM model
torch.save(model.state_dict(), 'lstm_model.pth')


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
from gensim.models import Word2Vec

# Tokenizer function
def tokenize(text):
    return text.lower().split()

# Prepare tokenized sentences for Word2Vec training
tokenized_statements = [tokenize(sentence) for sentence in data['statement']]

# Train Word2Vec model
embedding_dim = 200  # Set your preferred dimension size
word2vec_model = Word2Vec(sentences=tokenized_statements, vector_size=embedding_dim, window=5, min_count=1, workers=4)

# Generate a dictionary of Word2Vec embeddings
word2vec_embeddings = {word: torch.tensor(word2vec_model.wv[word], dtype=torch.float) for word in word2vec_model.wv.index_to_key}

# Encode each sentence using Word2Vec embeddings
def encode_sentence(sentence):
    tokens = tokenize(sentence)
    embeddings = [word2vec_embeddings[token] for token in tokens if token in word2vec_embeddings]
    return torch.stack(embeddings) if embeddings else torch.zeros((1, embedding_dim))

# Preprocess the data
data = data_cleaned[['statement', 'status']]
label_encoder = LabelEncoder()
data['status'] = label_encoder.fit_transform(data['status'])

# Apply encoding to each statement
data['embeddings'] = data['statement'].apply(encode_sentence)

# Split into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

# Custom Dataset and DataLoader setup
class TextDataset(Dataset):
    def __init__(self, data):
        self.embeddings = data['embeddings']
        self.labels = data['status']

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

def collate_fn(batch):
    texts, labels = zip(*batch)
    texts_padded = pad_sequence(texts, batch_first=True)
    labels = torch.tensor(labels, dtype=torch.long)
    return texts_padded, labels

train_dataset = TextDataset(train_data)
test_dataset = TextDataset(test_data)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)




In [ ]:
# Define LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, output_dim, n_layers=1, bidirectional=True, dropout=0.5):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        hidden = torch.cat((hidden[-2], hidden[-1]), dim=1) if self.lstm.bidirectional else hidden[-1]
        hidden = self.dropout(hidden)
        output = self.fc(hidden)
        return output

# Model parameters
hidden_dim = 128
output_dim = len(label_encoder.classes_)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(embedding_dim, hidden_dim, output_dim).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Function to train the model
def train_model(model, train_loader, criterion, optimizer, num_epochs=5):
    model.train()

    total_loss = 0
    for texts, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        texts, labels = texts.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(texts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    return total_loss / len(train_loader)

# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for texts, labels in test_loader:
            texts, labels = texts.to(device), labels.to(device)
            outputs = model(texts)
            predictions = torch.argmax(outputs, dim=1)
            total_correct += (predictions == labels).sum().item()
            total_samples += labels.size(0)
    accuracy = total_correct / total_samples
    return accuracy

# Function to test the model on a sample input
def test_sample(model, text):
    model.eval()
    with torch.no_grad():
        embedding = encode_sentence(text).unsqueeze(0).to(device)
        output = model(embedding)
        prediction = torch.argmax(output, dim=1).item()
    return label_encoder.inverse_transform([prediction])[0]



num_epochs = 30
for epoch in range(num_epochs):
    train_loss = train_model(model, train_loader, criterion, optimizer,num_epochs)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}")

    # Evaluate on test data
    test_accuracy = evaluate_model(model, test_loader)
    print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Training Epoch 1: 100%|██████████| 1317/1317 [00:16<00:00, 78.03it/s]


Epoch [1/30], Loss: 0.9237
Test Accuracy: 71.26%


Training Epoch 2: 100%|██████████| 1317/1317 [00:17<00:00, 73.47it/s]


Epoch [2/30], Loss: 0.7312
Test Accuracy: 73.53%


Training Epoch 3: 100%|██████████| 1317/1317 [00:16<00:00, 80.26it/s]


...

Epoch [28/30], Loss: 0.2332
Test Accuracy: 75.75%


Training Epoch 29: 100%|██████████| 1317/1317 [00:16<00:00, 80.98it/s]


Epoch [29/30], Loss: 0.2384
Test Accuracy: 75.63%


Training Epoch 30: 100%|██████████| 1317/1317 [00:16<00:00, 81.26it/s]


Epoch [30/30], Loss: 0.2236
Test Accuracy: 75.31%


In [ ]:
# Sample test
sample_text = "Right now I feel passively suicidal. I do not feel like trying to kill myself, but I strongly hope I do not wake up the next day. I wish I was dead, but I do not have any plans in mind. I absolutely hate myself. I hate my life. I hate that I cannot see myself getting any better. I fucking hate it all. I have been on abilify for a month"
predicted_class = test_sample(model, sample_text)
print(f"Predicted Class for sample text: {predicted_class}")

Predicted Class for sample text: 2


In [ ]:
# Function to test the model on a sample input
def test_sample(model, text):
    model.eval()
    with torch.no_grad():
        embedding = encode_sentence(text).unsqueeze(0).to(device)
        output = model(embedding)
        prediction = torch.argmax(output, dim=1).item()
    predicted_label = label_encoder.inverse_transform([prediction])[0]
    return predicted_label

# Sample test
sample_text = "I just want to go. Some days feel like I am making progress but then I see something that makes me remember the kind of pain I inflected in the person I love the most in this world, and I just want to fucking die. I did not do these things on purpose, but they happened and I am not being given a second chance to make them right.My tarotist (yeah, yeah, I know) told me that I would be doing alright by January next year but I do not see it coming. I do not see my 22 years on this Earth. I had the best birthday by her side last year and knowing she will not be in this one, I want to peel off my skin, get infected by the environment and drop dead.Yesterday was the last drop, she is hanging out with her ex and some other friends of hers. Her ex does not live near so I am assuming they are staying in her house. I know the girl I love is having a hard time, due to having to hide from her family about her sexual orientation while she is already struggling with trying to become independent and getting out of her toxic household environment. it is hard for her but I wish she would seek out some comfort in me too, but in our situation it is understandable why she does not.She sent a final letter through the app we met telling that she could not be friends with me and to not look out for her, but I am weak as hell. I want to know about her life and see that she is doing well.And yeah, she also played her role into not making the relationship work out but, while I am trying to be logical about it, she is dealing with a lot right now. But I felt so bad in a given situation that, instead of helping out and making her feel at ease, I put my problems on top of hers and that cracked her up. She feels that she cannot trust me, I wished she would see the progresses I have made so far but I feel like it does not even matter anymore. The promises I made her, I could not live up to them when the moment most needed it.I was dealing with a lot of unresolved issues when I went to her hometown, and I feel like shit because I made promises with the extroverted version of myself in mind, but that 'me' had been sleeping/dead for over a year (this pandemic sucked out all my social skills, fucking hell). I could not project myself in the way I knew I had in myself some time ago, I was not ready.I am trying to do better but I want to be seen by her. I never liked the idea of living just for myself, filling myself up with projects that mean little to nothing by the time I go to bed, it is exhausting.We talked about getting married, her coming to my college graduation, moving in with each other, we even made bets in relation to our future kids together. I feel empty as fuck and I cannot take it anymore. she is so beautiful, virtues and flaws, and I was so fucking stupid of making her feel like she should change things about herself when she is already perfect in my eyes, well, perfectly imperfect really. I always set high expectations for myself (me never feeling enough and some other thing related to my family relationships) and I put those same expectations on her, without realizing that those made me feel awful as well.do not go on and tell me ""there are plenty of other fish in the sea"". I do not fucking care about being romantic with anyone else, it makes me sick to even think about it.When she talks even concrete walls fall in love with her, she is that kind of person, and I took her for granted.I do not think I am going to make it to the next year. While I am not alone and I have a better relationship with my parents and some others, they will never be enough. I am in my hometown right now, staying at my dad's. I even told my psychologist about my suicide plan like a month ago. I know they did everything in their power to help me get out of this mental state but that is it, while I do appreciate them for what they tried to do. But now I just want to die in an accident because I cannot do it on my own and maybe it is better, my family will not know that I intentionally put myself through that anyway. At least I will be a donnor and somebody else will get a chance to live a life they want.I feel so fucking stupid and angry at myself.there are a lot of other things missing here but the post is long enough already. If you made it this far, thank you, you are kind, or just bored, heck if I know.Good luck and take care of your loved ones, do not ever take them for granted, do not be like me. I want to die and stop breathing life into my body"
predicted_class = test_sample(model, sample_text)
print(f"Predicted Class for sample text: {predicted_class}")

Predicted Class for sample text: 6


In [ ]:
label_encoder.classes_

array([0, 1, 2, 3, 4, 5, 6])